In [3]:
# With db
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import sqlite3
import re
import pyperclip

class PromptApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Prompt Generator")

        # Add menu
        self.menu_bar = tk.Menu(root)
        self.root.config(menu=self.menu_bar)

        self.prompt_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Menu", menu=self.prompt_menu)
        self.prompt_menu.add_command(label="Generate Prompt", command=self.show_generate_prompt)
        self.prompt_menu.add_command(label="Suggest Prompt for Leo", command=self.show_suggest_prompt)

        # Create frames for different interfaces
        self.frame_generate_prompt = tk.Frame(root)
        self.frame_suggest_prompt = tk.Frame(root)

        # Initialize both frames
        self.init_generate_prompt()
        self.init_suggest_prompt()

        # Start with the Generate Prompt interface by default
        self.show_generate_prompt()

    def init_generate_prompt(self):
        frame = self.frame_generate_prompt

        # Add logo
        self.logo_image = Image.open("Cáo.jpg")  # Thay thế bằng đường dẫn đến hình ảnh của bạn
        self.logo_image = self.logo_image.resize((100, 100), Image.LANCZOS)  # Thay đổi kích thước hình ảnh nếu cần
        self.logo_photo = ImageTk.PhotoImage(self.logo_image)

        self.logo_label = tk.Label(frame, image=self.logo_photo)
        self.logo_label.image = self.logo_photo  # Giữ tham chiếu tới hình ảnh
        self.logo_label.place(x=10, y=10)  # Đặt logo ở góc trên bên trái

        # Fetch prompt data from database
        self.prompts = self.fetch_prompts()
        self.prompt_titles = [prompt[0] for prompt in self.prompts]
        self.prompt_contents = [prompt[1] for prompt in self.prompts]

        # GUI Elements
        self.label_select = tk.Label(frame, text="Chọn prompt:")
        self.label_select.pack(pady=5)

        self.prompt_var = tk.StringVar()
        self.combobox = ttk.Combobox(frame, textvariable=self.prompt_var)
        self.combobox['values'] = self.prompt_titles
        self.combobox.current(0)
        self.combobox.pack(pady=5)
        self.combobox.bind("<<ComboboxSelected>>", self.update_label_input)

        self.entry_labels = []
        self.entry_fields = []
        self.entry_frame = tk.Frame(frame)
        self.entry_frame.pack(pady=5)

        self.button_generate = tk.Button(frame, text="Tạo Prompt", command=self.generate_prompt)
        self.button_generate.pack(pady=5)

        self.button_copy = tk.Button(frame, text="Copy", command=self.copy_prompt)
        self.button_copy.pack(pady=5)

        self.result = tk.Text(frame, wrap='word', width=100, height=30)
        self.result.pack(pady=5)

        self.update_label_input()

    def init_suggest_prompt(self):
        frame = self.frame_suggest_prompt

        self.label_submit_title = tk.Label(frame, text="Tiêu đề Prompt mới:")
        self.label_submit_title.pack(pady=5)
        self.entry_submit_title = tk.Entry(frame, width=50)
        self.entry_submit_title.pack(pady=5)

        self.label_submit_content = tk.Label(frame, text="Nội dung Prompt mới:")
        self.label_submit_content.pack(pady=5)
        self.entry_submit_content = tk.Text(frame, wrap='word', width=50, height=10)
        self.entry_submit_content.pack(pady=5)

        self.button_submit = tk.Button(frame, text="Gửi Prompt mới", command=self.submit_prompt)
        self.button_submit.pack(pady=5)

    def show_generate_prompt(self):
        self.frame_suggest_prompt.pack_forget()
        self.frame_generate_prompt.pack(fill='both', expand=True)

    def show_suggest_prompt(self):
        self.frame_generate_prompt.pack_forget()
        self.frame_suggest_prompt.pack(fill='both', expand=True)

    def fetch_prompts(self):
        conn = sqlite3.connect('prompts.db')
        c = conn.cursor()
        c.execute('SELECT title, content FROM prompts')
        prompts = c.fetchall()
        conn.close()
        return prompts

    def update_label_input(self, event=None):
        for label in self.entry_labels:
            label.destroy()
        for entry in self.entry_fields:
            entry.destroy()

        self.entry_labels = []
        self.entry_fields = []

        selected_index = self.combobox.current()
        selected_prompt = self.prompt_contents[selected_index]
        placeholders = re.findall(r'\[([^\]]+)\]', selected_prompt)
        unique_placeholders = list(dict.fromkeys(placeholders))

        for placeholder in unique_placeholders:
            label = tk.Label(self.entry_frame, text=f"Nhập {placeholder}:")
            label.pack(pady=2)
            entry = tk.Entry(self.entry_frame, width=50)
            entry.pack(pady=2)
            self.entry_labels.append(label)
            self.entry_fields.append(entry)

    def generate_prompt(self):
        selected_index = self.combobox.current()
        selected_prompt = self.prompt_contents[selected_index]
        placeholders = re.findall(r'\[([^\]]+)\]', selected_prompt)
        unique_placeholders = list(dict.fromkeys(placeholders))

        for placeholder, entry in zip(unique_placeholders, self.entry_fields):
            replacement = entry.get()
            selected_prompt = selected_prompt.replace(f"[{placeholder}]", replacement)

        self.result.delete(1.0, tk.END)
        self.result.insert(tk.END, selected_prompt)

    def copy_prompt(self):
        prompt_text = self.result.get(1.0, tk.END).strip()
        pyperclip.copy(prompt_text)

    def submit_prompt(self):
        title = self.entry_submit_title.get().strip()
        content = self.entry_submit_content.get("1.0", tk.END).strip()
        if title and content:
            conn = sqlite3.connect('prompts.db')
            c = conn.cursor()
            c.execute('INSERT INTO submitted_prompts (title, content) VALUES (?, ?)', (title, content))
            conn.commit()
            conn.close()
            self.entry_submit_title.delete(0, tk.END)
            self.entry_submit_content.delete("1.0", tk.END)
            messagebox.showinfo("Thông báo", "Prompt của bạn sẽ được cân nhắc kĩ lưỡng trước khi có trong hệ thống, Leo cảm ơn sự đóng góp của bạn.")
        else:
            messagebox.showwarning("Cảnh báo", "Vui lòng điền đầy đủ thông tin.")

if __name__ == "__main__":
    root = tk.Tk()
    app = PromptApp(root)
    root.mainloop()
